In [1]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeThreeModelsByScores

##Best config is 180: {'topK': 494, 'alpha': 0.1597863887003789}

In [2]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()
URM_stacked = combine_matrices(ICM=ICM, URM=URM)

In [3]:
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_stacked, train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage=0.85

EvaluatorHoldout: Ignoring 13924 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13908 ( 0.5%) Users that have less than 1 test interactions


In [6]:
##SELECT 25% OF USERS WITH LONGEST PROFILES TO FINE-TUNE THE HYPERPARAMETERS
import numpy as np
users_to_ignore=[]
profile_length = np.ediff1d(URM_train.indptr)
block_size = int(len(profile_length) * 0.25)

start_pos = 3 * block_size
end_pos = min(4 * block_size, len(profile_length))
sorted_users = np.argsort(profile_length)

users_in_group = sorted_users[start_pos:end_pos]

users_in_group_p_len = profile_length[users_in_group]
sorted_users = np.argsort(profile_length)

users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)

In [7]:
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10],ignore_users=sorted_users[users_not_in_group_flag])

EvaluatorHoldout: Ignoring 13924 ( 0.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13908 ( 0.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 10488 Users


In [8]:
stacked_train = combine_matrices(ICM=ICM, URM= URM_train)

In [9]:
### RP3Beta Recommender
RP3Beta_recommender = RP3betaRecommender(URM_train=URM_train)
RP3Beta_recommender.fit(topK=82, alpha=0.39354058844901263, beta=0.5811550127865287, implicit=True, normalize_similarity=True)

In [10]:
##SLIMELASTICNET Recommender
SLIME_recommender = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
SLIME_recommender.fit(topK=420, l1_ratio=0.06061216785568925, alpha=0.00554982938879437, workers = 10)

100%|████████████████████████████████████▉| 18000/18059 [14:00<00:01, 30.58it/s]

In [11]:
similarity_rp3 = RP3Beta_recommender.W_sparse
similarity_slime = SLIME_recommender.W_sparse

In [ ]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from skopt.space import Real,Integer

hyperparameters_range_dictionary = {
    "topK": Integer(450,600),
    "alpha": Real(0.0,0.3)
}

recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[stacked_train, similarity_rp3, similarity_slime],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS=[URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS={},
    FIT_POSITIONAL_ARGS=[],
    FIT_KEYWORD_ARGS={}
)

tuning_class = SearchBayesianSkopt(recommender_class=ItemKNNSimilarityHybridRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 200
n_random_starts = n_cases * 0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize=10,
                    n_cases=n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root=ItemKNNSimilarityHybridRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )




Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 559, 'alpha': 0.289860121567512}
EvaluatorHoldout: Processed 3496 (100.0%) in 4.05 sec. Users per second: 863
SearchBayesianSkopt: New best config found. Config 0: {'topK': 559, 'alpha': 0.289860121567512} - results: PRECISION: 0.2794050, PRECISION_RECALL_MIN_DEN: 0.2794050, RECALL: 0.0319441, MAP: 0.1410463, MAP_MIN_DEN: 0.1410463, MRR: 0.5055805, NDCG: 0.2836226, F1: 0.0573334, HIT_RATE: 0.9562357, ARHR_ALL_HITS: 0.8421283, NOVELTY: 0.0057215, AVERAGE_POPULARITY: 0.4658331, DIVERSITY_MEAN_INTER_LIST: 0.9437511, DIVERSITY_HERFINDAHL: 0.9943481, COVERAGE_ITEM: 0.0849438, COVERAGE_ITEM_CORRECT: 0.0600255, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9562357, DIVERSITY_GINI: 0.0183259, SHANNON_ENTROPY: 8.5867205, RATIO_DIVERSITY_HERFINDAHL: 0.9947153, RATIO_DIVERSITY_GINI: 0.0685924, RATIO_SHANNON_ENTROPY: 0.6880489, RATIO_AVERAGE_POPULARITY: 2.4016107, RATIO_NOVELTY: 0.03021

100%|████████████████████████████████████▉| 18056/18059 [14:10<00:00, 30.58it/s]

EvaluatorHoldout: Processed 13924 (100.0%) in 10.98 sec. Users per second: 1268
SearchBayesianSkopt: Config evaluated with evaluator_test. Config: {'topK': 559, 'alpha': 0.289860121567512} - results:
CUTOFF: 10 - PRECISION: 0.2798908, PRECISION_RECALL_MIN_DEN: 0.2844007, RECALL: 0.0703127, MAP: 0.1466355, MAP_MIN_DEN: 0.1487832, MRR: 0.5155798, NDCG: 0.2902061, F1: 0.1123911, HIT_RATE: 0.9385234, ARHR_ALL_HITS: 0.8628001, NOVELTY: 0.0057470, AVERAGE_POPULARITY: 0.4688183, DIVERSITY_MEAN_INTER_LIST: 0.9348190, DIVERSITY_HERFINDAHL: 0.9934752, COVERAGE_ITEM: 0.1941414, COVERAGE_ITEM_CORRECT: 0.1106927, COVERAGE_USER: 0.9957094, COVERAGE_USER_CORRECT: 0.9344966, DIVERSITY_GINI: 0.0226039, SHANNON_ENTROPY: 8.6634411, RATIO_DIVERSITY_HERFINDAHL: 0.9938420, RATIO_DIVERSITY_GINI: 0.0846047, RATIO_SHANNON_ENTROPY: 0.6941965, RATIO_AVERAGE_POPULARITY: 2.4170008, RATIO_NOVELTY: 0.0303508, 


SearchBayesianSkopt: Saving model in logs/ItemKNNSimilarityHybridRecommender

ItemKNNSimilarityHybridReco

100%|████████████████████████████████████▉| 18056/18059 [14:47<00:00, 20.35it/s]


EvaluatorHoldout: Processed 3496 (100.0%) in 3.68 sec. Users per second: 950
SearchBayesianSkopt: Config 4 is suboptimal. Config: {'topK': 530, 'alpha': 0.2529444531648216} - results: PRECISION: 0.2790332, PRECISION_RECALL_MIN_DEN: 0.2790332, RECALL: 0.0318739, MAP: 0.1413380, MAP_MIN_DEN: 0.1413380, MRR: 0.5069856, NDCG: 0.2837820, F1: 0.0572124, HIT_RATE: 0.9550915, ARHR_ALL_HITS: 0.8438534, NOVELTY: 0.0057257, AVERAGE_POPULARITY: 0.4616899, DIVERSITY_MEAN_INTER_LIST: 0.9466355, DIVERSITY_HERFINDAHL: 0.9946365, COVERAGE_ITEM: 0.0843347, COVERAGE_ITEM_CORRECT: 0.0597486, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9550915, DIVERSITY_GINI: 0.0185367, SHANNON_ENTROPY: 8.6239473, RATIO_DIVERSITY_HERFINDAHL: 0.9950037, RATIO_DIVERSITY_GINI: 0.0693814, RATIO_SHANNON_ENTROPY: 0.6910319, RATIO_AVERAGE_POPULARITY: 2.3802504, RATIO_NOVELTY: 0.0302382, 

Iteration No: 5 ended. Evaluation done at random point.
Time taken: 4.4166
Function value obtained: -0.1413
Current minimum: -0.1440
It

...

EvaluatorHoldout: Processed 3496 (100.0%) in 3.77 sec. Users per second: 928
SearchBayesianSkopt: Config 165 is suboptimal. Config: {'topK': 537, 'alpha': 0.004879260593285973} - results: PRECISION: 0.2828375, PRECISION_RECALL_MIN_DEN: 0.2828375, RECALL: 0.0323619, MAP: 0.1445364, MAP_MIN_DEN: 0.1445364, MRR: 0.5153377, NDCG: 0.2883620, F1: 0.0580785, HIT_RATE: 0.9605263, ARHR_ALL_HITS: 0.8596118, NOVELTY: 0.0057601, AVERAGE_POPULARITY: 0.4326822, DIVERSITY_MEAN_INTER_LIST: 0.9612988, DIVERSITY_HERFINDAHL: 0.9961024, COVERAGE_ITEM: 0.0855529, COVERAGE_ITEM_CORRECT: 0.0598593, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9605263, DIVERSITY_GINI: 0.0200257, SHANNON_ENTROPY: 8.8359935, RATIO_DIVERSITY_HERFINDAHL: 0.9964702, RATIO_DIVERSITY_GINI: 0.0749550, RATIO_SHANNON_ENTROPY: 0.7080230, RATIO_AVERAGE_POPULARITY: 2.2307008, RATIO_NOVELTY: 0.0304198, 

Iteration No: 166 ended. Search finished for the next optimal point.
Time taken: 6.4593
Function value obtained: -0.1445
Current mi

SearchBayesianSkopt: Config 172 is suboptimal. Config: {'topK': 538, 'alpha': 0.042071548760071596} - results: PRECISION: 0.2825229, PRECISION_RECALL_MIN_DEN: 0.2825229, RECALL: 0.0323326, MAP: 0.1440914, MAP_MIN_DEN: 0.1440914, MRR: 0.5133859, NDCG: 0.2877745, F1: 0.0580247, HIT_RATE: 0.9613844, ARHR_ALL_HITS: 0.8570180, NOVELTY: 0.0057551, AVERAGE_POPULARITY: 0.4367358, DIVERSITY_MEAN_INTER_LIST: 0.9598066, DIVERSITY_HERFINDAHL: 0.9959532, COVERAGE_ITEM: 0.0852760, COVERAGE_ITEM_CORRECT: 0.0600808, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9613844, DIVERSITY_GINI: 0.0197939, SHANNON_ENTROPY: 8.8097993, RATIO_DIVERSITY_HERFINDAHL: 0.9963210, RATIO_DIVERSITY_GINI: 0.0740872, RATIO_SHANNON_ENTROPY: 0.7059241, RATIO_AVERAGE_POPULARITY: 2.2515992, RATIO_NOVELTY: 0.0303933, 

Iteration No: 173 ended. Search finished for the next optimal point.
Time taken: 6.4475
Function value obtained: -0.1441
Current minimum: -0.1446
Iteration No: 174 started. Searching for the next optimal poin

Iteration No: 180 ended. Search finished for the next optimal point.
Time taken: 6.7063
Function value obtained: -0.1432
Current minimum: -0.1446
Iteration No: 181 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 589, 'alpha': 0.01380477214087984}
EvaluatorHoldout: Processed 3496 (100.0%) in 3.76 sec. Users per second: 930
SearchBayesianSkopt: Config 180 is suboptimal. Config: {'topK': 589, 'alpha': 0.01380477214087984} - results: PRECISION: 0.2831522, PRECISION_RECALL_MIN_DEN: 0.2831522, RECALL: 0.0324143, MAP: 0.1444767, MAP_MIN_DEN: 0.1444767, MRR: 0.5151730, NDCG: 0.2884404, F1: 0.0581696, HIT_RATE: 0.9616705, ARHR_ALL_HITS: 0.8592333, NOVELTY: 0.0057586, AVERAGE_POPULARITY: 0.4337633, DIVERSITY_MEAN_INTER_LIST: 0.9609449, DIVERSITY_HERFINDAHL: 0.9960670, COVERAGE_ITEM: 0.0856083, COVERAGE_ITEM_CORRECT: 0.0598593, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9616705, DIVERSITY_GINI: 0.0199621, SHANNON_ENTROPY: 8.8293139, RATIO_DIVER

EvaluatorHoldout: Processed 3496 (100.0%) in 3.89 sec. Users per second: 899
SearchBayesianSkopt: Config 187 is suboptimal. Config: {'topK': 482, 'alpha': 0.23454993417847853} - results: PRECISION: 0.2791476, PRECISION_RECALL_MIN_DEN: 0.2791476, RECALL: 0.0318954, MAP: 0.1418875, MAP_MIN_DEN: 0.1418875, MRR: 0.5096182, NDCG: 0.2844883, F1: 0.0572495, HIT_RATE: 0.9562357, ARHR_ALL_HITS: 0.8476278, NOVELTY: 0.0057288, AVERAGE_POPULARITY: 0.4592601, DIVERSITY_MEAN_INTER_LIST: 0.9481716, DIVERSITY_HERFINDAHL: 0.9947900, COVERAGE_ITEM: 0.0845008, COVERAGE_ITEM_CORRECT: 0.0599147, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9562357, DIVERSITY_GINI: 0.0186995, SHANNON_ENTROPY: 8.6460371, RATIO_DIVERSITY_HERFINDAHL: 0.9951574, RATIO_DIVERSITY_GINI: 0.0699911, RATIO_SHANNON_ENTROPY: 0.6928019, RATIO_AVERAGE_POPULARITY: 2.3677235, RATIO_NOVELTY: 0.0302544, 

Iteration No: 188 ended. Search finished for the next optimal point.
Time taken: 6.9931
Function value obtained: -0.1419
Current min

SearchBayesianSkopt: Config 194 is suboptimal. Config: {'topK': 597, 'alpha': 0.10830928569978233} - results: PRECISION: 0.2814645, PRECISION_RECALL_MIN_DEN: 0.2814645, RECALL: 0.0321953, MAP: 0.1437654, MAP_MIN_DEN: 0.1437654, MRR: 0.5146715, NDCG: 0.2873117, F1: 0.0577813, HIT_RATE: 0.9570938, ARHR_ALL_HITS: 0.8573786, NOVELTY: 0.0057465, AVERAGE_POPULARITY: 0.4440240, DIVERSITY_MEAN_INTER_LIST: 0.9565516, DIVERSITY_HERFINDAHL: 0.9956278, COVERAGE_ITEM: 0.0846116, COVERAGE_ITEM_CORRECT: 0.0602470, COVERAGE_USER: 1.0000000, COVERAGE_USER_CORRECT: 0.9570938, DIVERSITY_GINI: 0.0194808, SHANNON_ENTROPY: 8.7636040, RATIO_DIVERSITY_HERFINDAHL: 0.9959954, RATIO_DIVERSITY_GINI: 0.0729151, RATIO_SHANNON_ENTROPY: 0.7022225, RATIO_AVERAGE_POPULARITY: 2.2891734, RATIO_NOVELTY: 0.0303482, 

Iteration No: 195 ended. Search finished for the next optimal point.
Time taken: 6.8479
Function value obtained: -0.1438
Current minimum: -0.1446
Iteration No: 196 started. Searching for the next optimal point